# BERT

**NOTE**: given that this implementation is different from the original one in Tensorflow, some results can be slighty different.


Let's GO!

Required Imports.

In [1]:
import time

start_time = time.time()
!pip install transformers==4.3.2
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install sentencepiece

##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 41.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


GroupViT models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version.Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.
TAPAS models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version. Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.


In [2]:
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA L4


### Input Parameters


In [3]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 64
batch_size = 64

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator,
# each of the size of the output space
num_hidden_layers_g = 1;
# number of hidden layers in the discriminator,
# each of the size of the input space
num_hidden_layers_d = 1;
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.1

# Replicate labeled data to balance poorly represented datasets,
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
epsilon = 1e-8
num_train_epochs = 30
multi_gpu = True
# Scheduler
apply_scheduler = False
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------
# Since this version is compatible with Huggingface transformers, you can uncomment
# (or add) transformer models compatible with GAN

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2"
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"

#--------------------------------
#  Retrieve the TREC QC Dataset
#--------------------------------
! git clone https://github.com/mauriciokonrath/ganbert.git

#  NOTE: in this setting 50 classes are involved
labeled_file = "./ganbert/data/standardized_labeled_monsanto_withoutSub.tsv"
unlabeled_file = "./ganbert/data/unlabeled_enron_5000.tsv"
test_filename = "./ganbert/data/standardized_test_monsanto_withoutSub.tsv"

#categorias de rótulos que o modelo deve aprender a classificar.
label_list = ["UNK_UNK","GHOST_ghost", "TOXIC_toxic",
              "CHEMI_chemi", "REGUL_regul"]

Cloning into 'ganbert'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 185 (delta 111), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (185/185), 9.87 MiB | 5.08 MiB/s, done.
Resolving deltas: 100% (111/111), done.


Load the Tranformer Model

In [4]:
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.48.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/model.safetensors
A pretrained model of type `BertModel` contains parameters that have been renamed internally (a few are listed below but more are present in the model):
* `bert.embeddings.LayerNorm.beta` -> `bert.embeddings.LayerNorm.bias`
* `bert.embeddings.LayerNorm.gamma` -> `bert.embeddings.LayerNorm.weight`
If you are using a model from the Hub, consider submitting a PR to adjust these weights and help future users.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from th

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.48.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gr

Function required to load the dataset

In [5]:
def get_qc_examples(input_file):
  """Creates examples for the training and dev sets."""
  examples = []

  with open(input_file, 'r') as f:
      contents = f.read()
      file_as_list = contents.splitlines()
      for line in file_as_list[1:]:
          split = line.split(" ")
          question = ' '.join(split[1:])

          text_a = question
          inn_split = split[0].split(":")
          label = inn_split[0] + "_" + inn_split[1]
          examples.append((text_a, label))
      f.close()

  return examples

**Load** the input QC dataset (fine-grained)

In [6]:
#Load the examples
labeled_examples = get_qc_examples(labeled_file)
#unlabeled_examples = get_qc_examples(unlabeled_file)
#test_examples = get_qc_examples(test_filename)

Functions required to convert examples into Dataloader

In [7]:
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask:
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples):
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))

  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)

  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids)
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset),
              batch_size = batch_size) # Trains with this batch size.

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Convert the input examples into DataLoader

In [8]:
# Criar o mapeamento de labels
label_map = {}
for (i, label) in enumerate(label_list):
    label_map[label] = i

# ------------------------------
#   Carregar o conjunto de treinamento
# ------------------------------
train_examples = labeled_examples

# Normalizar os rótulos em train_examples
train_examples = [(text[0], ' '.join(text[1].split()).replace("\t", "_")) for text in train_examples]

# Atualizar label_map dinamicamente
dataset_labels = set(example[1] for example in train_examples)
for label in dataset_labels:
    if label not in label_map:
        label_map[label] = len(label_map)

print("Updated label_map:", label_map)

# Gerar a máscara para os rótulos de treinamento
train_label_masks = np.ones(len(train_examples), dtype=bool)

# Gerar o dataloader novamente
train_dataloader = generate_data_loader(
    train_examples,
    train_label_masks,
    label_map,
    do_shuffle=True,
    balance_label_examples=apply_balance
)

# ------------------------------
#   Carregar o conjunto de teste (opcional)
# ------------------------------
# O dataset de teste (se necessário) pode ser processado de forma similar
# test_label_masks = np.ones(len(test_examples), dtype=bool)
# test_dataloader = generate_data_loader(
#     test_examples,
#     test_label_masks,
#     label_map,
#     do_shuffle=False,
#     balance_label_examples=False
# )


Updated label_map: {'UNK_UNK': 0, 'GHOST_ghost': 1, 'TOXIC_toxic': 2, 'CHEMI_chemi': 3, 'REGUL_regul': 4, 'CHEMI_chemi These': 5, 'GHOST_ghost In': 6, 'REGUL_regul In': 7, 'REGUL_regul This': 8, 'TOXIC_toxic This': 9, 'CHEMI_chemi This': 10, 'REGUL_regul “In': 11, 'TOXIC_toxic In': 12, 'GHOST_ghost This': 13}


<ipython-input-7-16c38d5f7491>:54: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  label_mask_array = torch.tensor(label_mask_array)


Let's go with the training procedure

In [9]:
import time
import torch
import numpy as np
from transformers import *
from sklearn.metrics import confusion_matrix  # Importa a função para calcular a matriz de confusão

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Inicialização do modelo
transformer = transformer.to(device)
transformer.train()

# Otimizador
optimizer = torch.optim.AdamW(transformer.parameters(), lr=5e-5)

# Função para calcular métricas (precision, recall e F1 Score)
def calculate_metrics(y_true, y_pred):
    # Classes únicas
    classes = np.unique(y_true)

    recalls = []
    precisions = []
    f1_scores = []

    for cls in classes:
        # True Positives, False Positives, False Negatives
        tp = np.sum((y_true == cls) & (y_pred == cls))
        fp = np.sum((y_true != cls) & (y_pred == cls))
        fn = np.sum((y_true == cls) & (y_pred != cls))

        # Precision
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0

        # Recall
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        # F1 Score
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1_score)

    # Média das métricas por classe
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    return avg_precision, avg_recall, avg_f1

# Loop de treinamento
training_stats = []
total_t0 = time.time()

# Inicializar listas para armazenar as previsões e os rótulos finais
final_preds = []
final_labels = []

for epoch_i in range(0, num_train_epochs):
    print(f"======== Epoch {epoch_i + 1} / {num_train_epochs} ========")
    print("Training...")

    t0 = time.time()
    tr_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        # Utilizando o embedding do primeiro token (por exemplo, [CLS]) para classificação
        logits = outputs.last_hidden_state[:, 0, :]
        loss_fn = torch.nn.CrossEntropyLoss()
        loss = loss_fn(logits, b_labels)

        loss.backward()
        optimizer.step()

        tr_loss += loss.item()

        # Cálculo da acurácia
        preds = torch.argmax(logits, dim=1)
        correct_predictions += (preds == b_labels).sum().item()
        total_predictions += b_labels.size(0)

        # Armazenar previsões e rótulos para avaliação final
        final_preds.extend(preds.detach().cpu())
        final_labels.extend(b_labels.detach().cpu())

    # Cálculo do loss médio e acurácia
    avg_train_loss = tr_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions

    print(f"  Training Accuracy: {train_accuracy:.3f}")
    print(f"  Average Training Loss: {avg_train_loss:.3f}")

    # Salvar estatísticas da época
    training_stats.append({
        'epoch': epoch_i + 1,
        'Training Loss': avg_train_loss,
        'Training Accuracy': train_accuracy,
    })

# Avaliação final com cálculo das métricas adicionais
print("\nEvaluating the model...")

# Conversão das listas para arrays NumPy
final_preds = torch.stack(final_preds).numpy()
final_labels = torch.stack(final_labels).numpy()

# Exibe os valores únicos encontrados nos rótulos e nas previsões
print("Unique labels in final_labels:", np.unique(final_labels))
print("Unique predictions in final_preds:", np.unique(final_preds))

# Cálculo das métricas: precisão, recall e F1 Score
precision, recall, f1_score = calculate_metrics(final_labels, final_preds)
# Cálculo da matriz de confusão
conf_mat = confusion_matrix(final_labels, final_preds)

print("\nFinal Metrics:")
print(f"  Precision: {precision:.3f}")
print(f"  Recall: {recall:.3f}")
print(f"  F1 Score: {f1_score:.3f}")
print("  Confusion Matrix:")
print(conf_mat)

# Salvando o modelo treinado
model_save_path = "bert_model.pth"
torch.save(transformer.state_dict(), model_save_path)
print(f"\nModel saved to {model_save_path}")

# Exibindo estatísticas finais de treinamento
print("\nTraining complete.")
for stat in training_stats:
    print(f"Epoch {stat['epoch']}: Loss={stat['Training Loss']:.3f}, Accuracy={stat['Training Accuracy']:.3f}")

print(f"\nFinal Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1_score:.3f}")


Using device: cuda
======== Epoch 1 / 30 ========
Training...
  Training Accuracy: 0.000
  Average Training Loss: 7.496
======== Epoch 2 / 30 ========
Training...
  Training Accuracy: 0.000
  Average Training Loss: 6.575
======== Epoch 3 / 30 ========
Training...
  Training Accuracy: 0.175
  Average Training Loss: 6.172
======== Epoch 4 / 30 ========
Training...
  Training Accuracy: 0.200
  Average Training Loss: 5.791
======== Epoch 5 / 30 ========
Training...
  Training Accuracy: 0.400
  Average Training Loss: 5.283
======== Epoch 6 / 30 ========
Training...
  Training Accuracy: 0.475
  Average Training Loss: 4.637
======== Epoch 7 / 30 ========
Training...
  Training Accuracy: 0.325
  Average Training Loss: 4.001
======== Epoch 8 / 30 ========
Training...
  Training Accuracy: 0.375
  Average Training Loss: 3.005
======== Epoch 9 / 30 ========
Training...
  Training Accuracy: 0.400
  Average Training Loss: 2.764
======== Epoch 10 / 30 ========
Training...
  Training Accuracy: 0.575
 

In [10]:
for stat in training_stats:
  print(stat)

print("\nTraining complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

resultado = sum(range(10**6))

end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time:.4f} segundos")

{'epoch': 1, 'Training Loss': 7.496011257171631, 'Training Accuracy': 0.0}
{'epoch': 2, 'Training Loss': 6.575135231018066, 'Training Accuracy': 0.0}
{'epoch': 3, 'Training Loss': 6.172366619110107, 'Training Accuracy': 0.175}
{'epoch': 4, 'Training Loss': 5.790643692016602, 'Training Accuracy': 0.2}
{'epoch': 5, 'Training Loss': 5.282961845397949, 'Training Accuracy': 0.4}
{'epoch': 6, 'Training Loss': 4.636702060699463, 'Training Accuracy': 0.475}
{'epoch': 7, 'Training Loss': 4.000504970550537, 'Training Accuracy': 0.325}
{'epoch': 8, 'Training Loss': 3.005439043045044, 'Training Accuracy': 0.375}
{'epoch': 9, 'Training Loss': 2.764087200164795, 'Training Accuracy': 0.4}
{'epoch': 10, 'Training Loss': 2.4126486778259277, 'Training Accuracy': 0.575}
{'epoch': 11, 'Training Loss': 2.2128689289093018, 'Training Accuracy': 0.55}
{'epoch': 12, 'Training Loss': 2.1473023891448975, 'Training Accuracy': 0.65}
{'epoch': 13, 'Training Loss': 1.6109384298324585, 'Training Accuracy': 0.675}
{'e